In [1]:
import os
os.chdir(r"C:\Users\xciur\OneDrive\Escriptori\UOC - cIENCIA DE dADES\4 semestre\visualització de dades\pac3")

In [2]:
#Actualització arxiu rutes Geojson
import json
import pandas as pd

#carguem arxiu coordenades amnuals
excel_file = 'coordenades_manuals.xlsx'

# leccutra arxiu
df = pd.read_excel(excel_file)
df['municipi'] = df['municipi'].str.strip()

#Arxiu geojson a actualitzar
geojson_file = 'ruta_final.geojson'

#Carguem geojson
with open(geojson_file, 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

#Municipies existents en el geojson
municipis_existents = set(feature['properties'].get('Municipi') for feature in geojson_data['features'])

# Actualització coordenades i afegir ubicacions noves
for index, row in df.iterrows():
    municipi = row['municipi']
    # Si la ubicacio existeix, s'actualitzen coordenades
    if municipi in municipis_existents:
        for feature in geojson_data['features']:
            if feature['properties'].get('Municipi') == municipi:
                coords_string = row['coordenades']
                lat, lon = map(float, coords_string.split(', '))
                feature['geometry']['coordinates'] = [lon, lat]
    # Si el municipi no existeix, s'afegeix
    else:
        coords_string = row['coordenades']
        lat, lon = map(float, coords_string.split(', '))
        new_feature = {
            "type": "Feature",
            "properties": {"Municipi": municipi},
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat]
            }
        }
        geojson_data['features'].append(new_feature)

#Guardar arxiu actualitzat
updated_geojson_file = 'ruta_final_definitiva.geojson'
with open(updated_geojson_file, 'w', encoding='utf-8') as file:
    json.dump(geojson_data, file, ensure_ascii=False)